# Architektur Neuronales Netz

In [1]:
# aNN Architektur

# Importe / Bibliotheken
import torch
from torch import nn
from torch import log10
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.optim.lr_scheduler import StepLR, MultiStepLR, ReduceLROnPlateau
from sklearn.metrics import r2_score as r2
from sklearn.metrics import max_error
# from sklearn.metrics import mean_squared_error as MSE
# from sklearn.metrics import mean_absolute_error as MAE
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

#### Default Datentyp

In [2]:
torch.set_default_dtype(torch.float64)

#### Erzeugnung des Moduls

In [3]:
class NeuralNetwork(nn.Module):
    
    #Initalisierung der Netzwerk layers
    def __init__(self, input_size, hidden1_size, hidden2_size, hidden3_size, output_size):
    
        super().__init__() #Referenz zur Base Class (nn.Module)
        #Kaskade der Layer
        self.linear_afunc_stack = nn.Sequential(
            nn.BatchNorm1d(input_size), # Normalisierung, damit Inputdaten gleiche Größenordnung haben
            nn.Linear(input_size, hidden1_size), #Lineare Transformation mit gespeicherten weights und biases
            #nn.LayerNorm(hidden1_size),
            nn.Tanh(), #Nicht lineare Aktivierungsfunktion um komplexe nichtlineare Zusammenhänge abzubilden
            #nn.SELU(),
            nn.BatchNorm1d(hidden1_size),
            nn.Linear(hidden1_size, hidden2_size),
            #nn.LayerNorm(hidden2_size),
            nn.Tanh(),
            #nn.SELU(),
            nn.BatchNorm1d(hidden2_size),
            nn.Linear(hidden2_size, hidden3_size),
            #nn.LayerNorm(hidden3_size),
            nn.Tanh(),
            #nn.SELU(),
            nn.BatchNorm1d(hidden3_size),
            nn.Linear(hidden3_size, output_size),
        )

    #Implementierung der Operationen auf Input Daten
    def forward(self, x):
        out = self.linear_afunc_stack(x)
        return out

#### Ausgabe Modul

In [4]:
model = NeuralNetwork(5, 200, 200, 200, 1)
print(model)

NeuralNetwork(
  (linear_afunc_stack): Sequential(
    (0): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=5, out_features=200, bias=True)
    (2): Tanh()
    (3): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=200, out_features=200, bias=True)
    (5): Tanh()
    (6): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Linear(in_features=200, out_features=200, bias=True)
    (8): Tanh()
    (9): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Linear(in_features=200, out_features=1, bias=True)
  )
)


#### DataLoader

In [5]:
batch_size = 32 #Zahl der Datenpaare die vor einem erneuten Update der Parameter ins Netzt gegeben werden
eq_data_file = Path.cwd() / 'data' / 'eq_dataset.npz' #Import der GGW Daten

res = np.load(eq_data_file)

# Bei Speicherung wurden Daten als T, p, x_0 und xi gespeichert
# Inputs T, p, x_0[H2,N2,NH3]
# Outputs xi
# Umwandlen der np.arrays in torch.tensors zur besseren Arbeit mit PyTorch
T = torch.tensor(res['T'])
p = torch.tensor(res['p'])
x_0 = torch.tensor(res['x_0'])
xi = torch.tensor(res['xi'])

#Anpassen der Daten auf gleiche Größenordnung
#T = log10(T)
# T = T / 850
# p = p / 1000

# T = torch.tensor(res['T']).float()
# p = torch.tensor(res['p']).float()
# x_0 = torch.tensor(res['x_0']).float()
# xi = torch.tensor(res['xi']).float()

# print(T.dtype)
# print(xi.dtype)

x_input = torch.stack((T, p ,x_0[:,0],x_0[:,1],x_0[:,2]),1)
y_output = xi.reshape((-1,1))
#print(x_input.size())
# print(xi.size())

# Tensoren zu einem großen Set gruppieren
dataset = TensorDataset(x_input, y_output)

# for x,y in dataset:
#     print(x)
#     print(y)
    
# Split in Trainings und Test Set
train_dataset, test_dataset = random_split(dataset, 
                                           [int(0.8*len(dataset)), int(0.2*len(dataset))], # splitting 80/20
                                           generator = torch.Generator().manual_seed(42)) # Festlegung seed zur Reproduktivität

# Erzeugen der DataLoader zur Arbeit mit Daten
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True) # shuffle batches zur Reduzierung von overfitting
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)


#### Generierung Netzwerk, Festlegung von loss Funktion und Optimizer

In [6]:
# Erzeugung aNN
net = NeuralNetwork(5, 200, 200, 200, 1)

# Loss Funktion; gibt Fehler an
#loss_fn = nn.MSELoss()
loss_fn = nn.L1Loss()

#Definition custom loss Funktion, MRE
def MRELoss(outputs, targets):
    
    loss = torch.mean(abs((outputs - targets) / targets))
    
    return loss
    

#Optimizer
learning_rate = 1e-2
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)
#scheduler = StepLR(optimizer, step_size = 30, gamma = 0.1)
#scheduler = MultiStepLR(optimizer, milestones=[30, 70, 100], gamma = 0.1)
scheduler = ReduceLROnPlateau(optimizer, factor = 0.1, patience = 10, threshold = 1e-4)

In [7]:
# learning_rate = 1e-3
# optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate, )

#### Funktion zur Bestimmung der Genauigkeit

In [8]:
def check_accuracy(loader, net):
    
    loss = 0
    MRE = 0
    train_correct = 0
    train_total = len(loader.dataset)
    num_batches = len(loader) 
    #train_total = 0
    
    net.eval() # Put network in evaluation mode
    
    if loader == train_dataloader:
        dataset = "Train"
    else:
        dataset = "Test"
    
    with torch.no_grad():
        for X, y in loader:
            pred = net(X)
           
            #loss += MRELoss(pred, y).item()
            loss += loss_fn(pred, y) # Calculate the loss
            MRE += MRELoss(pred, y)

            # Record the correct predictions for training data
            #_, predictions = torch.max(pred.data, 1)
            for i in range(len(pred)):
                if pred[i] - y[i] <= 0.01:
                    train_correct += 1
            #train_correct += (abs(pred.argmax(1) - y) <= 0.01).sum().item()
            #train_correct += (abs(predictions - y.data) <= 0.01).sum()
            #train_total += predictions.size(0)
            
        # Genauigkeit berechnen
        acc = float(train_correct) / float(train_total) * 100
        acc = round(acc, 2)
        
        loss /= num_batches
        MRE /= num_batches
        print(f"{dataset} Error: \n Accuracy: {acc}%, Avg loss: {loss:>8f}, MRE: {MRE:>8f} \n")

    net.train()
    
    return acc, loss, MRE

#### Training

In [ ]:
num_epochs = 200 #Iterationen über Datenset

train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []
train_MRE = []
test_MRE = []

#Optimierungsloop
for epoch in range(num_epochs):
#     train_correct = 0
#     train_total = 0
        
    for batch, (X,y) in enumerate(train_dataloader):
        
#         print(X.shape)
#         print(X.dtype)
        
        net.train() #Trainingmodus
        
        # forward
        pred = net(X)  # Do the forward pass
        loss = loss_fn(pred, y) # Calculate the loss
        #loss = MRELoss(pred, y)
        
        # backward
        optimizer.zero_grad() # Clear off the gradients from any past operation
        loss.backward()       # Calculate the gradients with help of back propagation, updating weights and biases
        
        # adam step gradient descent
        optimizer.step()      # Ask the optimizer to adjust the parameters based on the gradients  

        print ('Epoch %d/%d, Iteration %d/%d, Loss: %.4f' 
               %(epoch+1, num_epochs, batch+1, len(train_dataset)//batch_size, loss.item()))
        
    
    #scheduler.step() # Reduzieren Learning Rate (falls step size erreicht)
    net.eval() # Put the network into evaluation mode
    
    # Book keeping    
    # What was our train accuracy?
    tr_acc, tr_loss, tr_MRE = check_accuracy(train_dataloader, net)
    
    #Record loss and accuracy
    train_accuracy.append(tr_acc)
    train_loss.append(tr_loss)
    train_MRE.append(tr_MRE)
    
    scheduler.step(tr_loss) # LR scheduler step für reduceonPlateau
    
    # How did we do on the test set (the unseen set)
    # Record the correct predictions for test data
    t_acc, t_loss, t_MRE = check_accuracy(test_dataloader, net)
    test_accuracy.append(t_acc)
    test_loss.append(t_loss)
    test_MRE.append(t_MRE)

Epoch 1/200, Iteration 1/25, Loss: 0.3538
Epoch 1/200, Iteration 2/25, Loss: 4.3141
Epoch 1/200, Iteration 3/25, Loss: 1.6414
Epoch 1/200, Iteration 4/25, Loss: 0.4782
Epoch 1/200, Iteration 5/25, Loss: 0.7025
Epoch 1/200, Iteration 6/25, Loss: 0.6295
Epoch 1/200, Iteration 7/25, Loss: 0.4898
Epoch 1/200, Iteration 8/25, Loss: 0.2715
Epoch 1/200, Iteration 9/25, Loss: 0.2758
Epoch 1/200, Iteration 10/25, Loss: 0.2750
Epoch 1/200, Iteration 11/25, Loss: 0.2792
Epoch 1/200, Iteration 12/25, Loss: 0.2454
Epoch 1/200, Iteration 13/25, Loss: 0.1645
Epoch 1/200, Iteration 14/25, Loss: 0.1991
Epoch 1/200, Iteration 15/25, Loss: 0.2268
Epoch 1/200, Iteration 16/25, Loss: 0.1952
Epoch 1/200, Iteration 17/25, Loss: 0.1317
Epoch 1/200, Iteration 18/25, Loss: 0.1559
Epoch 1/200, Iteration 19/25, Loss: 0.1445
Epoch 1/200, Iteration 20/25, Loss: 0.1496
Epoch 1/200, Iteration 21/25, Loss: 0.1763
Epoch 1/200, Iteration 22/25, Loss: 0.1314
Epoch 1/200, Iteration 23/25, Loss: 0.1103
Epoch 1/200, Iterati

Epoch 8/200, Iteration 20/25, Loss: 0.0359
Epoch 8/200, Iteration 21/25, Loss: 0.0343
Epoch 8/200, Iteration 22/25, Loss: 0.0369
Epoch 8/200, Iteration 23/25, Loss: 0.0347
Epoch 8/200, Iteration 24/25, Loss: 0.0378
Epoch 8/200, Iteration 25/25, Loss: 0.0379
Train Error: 
 Accuracy: 68.25%, Avg loss: 0.032861, MRE: 2.275302 

Test Error: 
 Accuracy: 64.0%, Avg loss: 0.034505, MRE: 6.741541 

Epoch 9/200, Iteration 1/25, Loss: 0.0353
Epoch 9/200, Iteration 2/25, Loss: 0.0283
Epoch 9/200, Iteration 3/25, Loss: 0.0507
Epoch 9/200, Iteration 4/25, Loss: 0.0583
Epoch 9/200, Iteration 5/25, Loss: 0.0414
Epoch 9/200, Iteration 6/25, Loss: 0.0648
Epoch 9/200, Iteration 7/25, Loss: 0.0491
Epoch 9/200, Iteration 8/25, Loss: 0.0688
Epoch 9/200, Iteration 9/25, Loss: 0.0765
Epoch 9/200, Iteration 10/25, Loss: 0.0614
Epoch 9/200, Iteration 11/25, Loss: 0.0579
Epoch 9/200, Iteration 12/25, Loss: 0.0518
Epoch 9/200, Iteration 13/25, Loss: 0.0509
Epoch 9/200, Iteration 14/25, Loss: 0.0730
Epoch 9/200, 

Train Error: 
 Accuracy: 61.5%, Avg loss: 0.035495, MRE: 1.788489 

Test Error: 
 Accuracy: 55.0%, Avg loss: 0.039538, MRE: 3.434883 

Epoch 16/200, Iteration 1/25, Loss: 0.0245
Epoch 16/200, Iteration 2/25, Loss: 0.0455
Epoch 16/200, Iteration 3/25, Loss: 0.0592
Epoch 16/200, Iteration 4/25, Loss: 0.0579
Epoch 16/200, Iteration 5/25, Loss: 0.0448
Epoch 16/200, Iteration 6/25, Loss: 0.0352
Epoch 16/200, Iteration 7/25, Loss: 0.0453
Epoch 16/200, Iteration 8/25, Loss: 0.0635
Epoch 16/200, Iteration 9/25, Loss: 0.0533
Epoch 16/200, Iteration 10/25, Loss: 0.0335
Epoch 16/200, Iteration 11/25, Loss: 0.0571
Epoch 16/200, Iteration 12/25, Loss: 0.0440
Epoch 16/200, Iteration 13/25, Loss: 0.0373
Epoch 16/200, Iteration 14/25, Loss: 0.0476
Epoch 16/200, Iteration 15/25, Loss: 0.0333
Epoch 16/200, Iteration 16/25, Loss: 0.0410
Epoch 16/200, Iteration 17/25, Loss: 0.0451
Epoch 16/200, Iteration 18/25, Loss: 0.0504
Epoch 16/200, Iteration 19/25, Loss: 0.0471
Epoch 16/200, Iteration 20/25, Loss: 0

Epoch 23/200, Iteration 5/25, Loss: 0.0430
Epoch 23/200, Iteration 6/25, Loss: 0.0314
Epoch 23/200, Iteration 7/25, Loss: 0.0397
Epoch 23/200, Iteration 8/25, Loss: 0.0418
Epoch 23/200, Iteration 9/25, Loss: 0.0420
Epoch 23/200, Iteration 10/25, Loss: 0.0450
Epoch 23/200, Iteration 11/25, Loss: 0.0311
Epoch 23/200, Iteration 12/25, Loss: 0.0473
Epoch 23/200, Iteration 13/25, Loss: 0.0647
Epoch 23/200, Iteration 14/25, Loss: 0.0504
Epoch 23/200, Iteration 15/25, Loss: 0.0250
Epoch 23/200, Iteration 16/25, Loss: 0.0307
Epoch 23/200, Iteration 17/25, Loss: 0.0395
Epoch 23/200, Iteration 18/25, Loss: 0.0387
Epoch 23/200, Iteration 19/25, Loss: 0.0361
Epoch 23/200, Iteration 20/25, Loss: 0.0194
Epoch 23/200, Iteration 21/25, Loss: 0.0262
Epoch 23/200, Iteration 22/25, Loss: 0.0326
Epoch 23/200, Iteration 23/25, Loss: 0.0316
Epoch 23/200, Iteration 24/25, Loss: 0.0248
Epoch 23/200, Iteration 25/25, Loss: 0.0540
Train Error: 
 Accuracy: 69.38%, Avg loss: 0.041289, MRE: 2.315426 

Test Error: 

Epoch 30/200, Iteration 1/25, Loss: 0.0347
Epoch 30/200, Iteration 2/25, Loss: 0.0250
Epoch 30/200, Iteration 3/25, Loss: 0.0261
Epoch 30/200, Iteration 4/25, Loss: 0.0318
Epoch 30/200, Iteration 5/25, Loss: 0.0383
Epoch 30/200, Iteration 6/25, Loss: 0.0359
Epoch 30/200, Iteration 7/25, Loss: 0.0403
Epoch 30/200, Iteration 8/25, Loss: 0.0283
Epoch 30/200, Iteration 9/25, Loss: 0.0286
Epoch 30/200, Iteration 10/25, Loss: 0.0213
Epoch 30/200, Iteration 11/25, Loss: 0.0344
Epoch 30/200, Iteration 12/25, Loss: 0.0517
Epoch 30/200, Iteration 13/25, Loss: 0.0267
Epoch 30/200, Iteration 14/25, Loss: 0.0285
Epoch 30/200, Iteration 15/25, Loss: 0.0301
Epoch 30/200, Iteration 16/25, Loss: 0.0266
Epoch 30/200, Iteration 17/25, Loss: 0.0200
Epoch 30/200, Iteration 18/25, Loss: 0.0215
Epoch 30/200, Iteration 19/25, Loss: 0.0293
Epoch 30/200, Iteration 20/25, Loss: 0.0265
Epoch 30/200, Iteration 21/25, Loss: 0.0339
Epoch 30/200, Iteration 22/25, Loss: 0.0314
Epoch 30/200, Iteration 23/25, Loss: 0.03

Test Error: 
 Accuracy: 67.5%, Avg loss: 0.018788, MRE: 0.897981 

Epoch 37/200, Iteration 1/25, Loss: 0.0262
Epoch 37/200, Iteration 2/25, Loss: 0.0162
Epoch 37/200, Iteration 3/25, Loss: 0.0242
Epoch 37/200, Iteration 4/25, Loss: 0.0241
Epoch 37/200, Iteration 5/25, Loss: 0.0296
Epoch 37/200, Iteration 6/25, Loss: 0.0206
Epoch 37/200, Iteration 7/25, Loss: 0.0350
Epoch 37/200, Iteration 8/25, Loss: 0.0239
Epoch 37/200, Iteration 9/25, Loss: 0.0155
Epoch 37/200, Iteration 10/25, Loss: 0.0244
Epoch 37/200, Iteration 11/25, Loss: 0.0176
Epoch 37/200, Iteration 12/25, Loss: 0.0124
Epoch 37/200, Iteration 13/25, Loss: 0.0158
Epoch 37/200, Iteration 14/25, Loss: 0.0177
Epoch 37/200, Iteration 15/25, Loss: 0.0225
Epoch 37/200, Iteration 16/25, Loss: 0.0286
Epoch 37/200, Iteration 17/25, Loss: 0.0168
Epoch 37/200, Iteration 18/25, Loss: 0.0258
Epoch 37/200, Iteration 19/25, Loss: 0.0262
Epoch 37/200, Iteration 20/25, Loss: 0.0470
Epoch 37/200, Iteration 21/25, Loss: 0.0258
Epoch 37/200, Iter

Train Error: 
 Accuracy: 73.0%, Avg loss: 0.018801, MRE: 0.898628 

Test Error: 
 Accuracy: 69.0%, Avg loss: 0.020273, MRE: 3.019184 

Epoch 44/200, Iteration 1/25, Loss: 0.0322
Epoch 44/200, Iteration 2/25, Loss: 0.0237
Epoch 44/200, Iteration 3/25, Loss: 0.0224
Epoch 44/200, Iteration 4/25, Loss: 0.0233
Epoch 44/200, Iteration 5/25, Loss: 0.0322
Epoch 44/200, Iteration 6/25, Loss: 0.0202
Epoch 44/200, Iteration 7/25, Loss: 0.0496
Epoch 44/200, Iteration 8/25, Loss: 0.0350
Epoch 44/200, Iteration 9/25, Loss: 0.0381
Epoch 44/200, Iteration 10/25, Loss: 0.0256
Epoch 44/200, Iteration 11/25, Loss: 0.0216
Epoch 44/200, Iteration 12/25, Loss: 0.0149
Epoch 44/200, Iteration 13/25, Loss: 0.0153
Epoch 44/200, Iteration 14/25, Loss: 0.0150
Epoch 44/200, Iteration 15/25, Loss: 0.0299
Epoch 44/200, Iteration 16/25, Loss: 0.0367
Epoch 44/200, Iteration 17/25, Loss: 0.0341
Epoch 44/200, Iteration 18/25, Loss: 0.0179
Epoch 44/200, Iteration 19/25, Loss: 0.0244
Epoch 44/200, Iteration 20/25, Loss: 0

#### Plots loss vs Epoch

In [ ]:
fig,ax = plt.subplots()
#fig.figsize=(12, 8)
ax.semilogy(train_loss, label='train loss')
ax.semilogy(test_loss, label='test loss')
plt.title("Train and Test Loss")
ax.set(xlabel = '$Epochs$ / 1', ylabel = '|$\\xi - \\xi\mathregular{_{pred}}$| / mol') #Beschriftung Achsen; Kursiv durch $$; Index durch _{}
ax.tick_params(direction = 'in') #, length = 20, width = 3)

plt.legend()
plt.tight_layout()
plt.show()

#### Parity Plot

In [ ]:
xi_real = []
xi_pred = []
for (X,y) in train_dataloader:
    xi_real = np.append(xi_real, y.numpy())
    xi_pred = np.append(xi_pred, net(X).detach().numpy())

xi_real_test = []
xi_pred_test = []
for (X,y) in test_dataloader:
    xi_real_test = np.append(xi_real_test, y.numpy())
    xi_pred_test = np.append(xi_pred_test, net(X).detach().numpy())

print('Training Dataset: R^2 =', r2(xi_real,xi_pred))
print('Test Dataset: R^2 =', r2(xi_real_test,xi_pred_test))
print('Max Error Training: |xi - xi_pred| =', max_error(xi_real, xi_pred))
print('Max Error Test: |xi - xi_pred| =', max_error(xi_real_test, xi_pred_test))

# find the boundaries of X and Y values
bounds = (min(xi_real.min(), xi_pred.min()) - int(0.1 * xi_pred.min()), max(xi_real.max(), xi_pred.max())+ int(0.1 * xi_pred.max()))

fig,ax = plt.subplots(1,2, figsize =(10,10))

# # Reset the limits
# ax[0] = plt.gca()
ax[0].set_xlim(bounds)
ax[0].set_ylim(bounds)
# Ensure the aspect ratio is square
ax[0].set_aspect("equal", adjustable="box")

ax[0].plot(xi_real, xi_pred, '.')
ax[0].plot([0, 1], [0, 1], "r-",lw=2 ,transform=ax[0].transAxes)
ax[0].set(xlabel = '$\\xi$ / mol', ylabel = '$\\xi\mathregular{_{pred}}$ / mol')
ax[0].tick_params(direction = 'in') #, length = 20, width = 3)
ax[0].set_title('Train Data')
#ax[0].legend(['$\\mathregular{R^2}$ = ', r2(xi_real,xi_pred)], markerscale=0)

# Reset the limits
#ax[1] = plt.gca()
ax[1].set_xlim(bounds)
ax[1].set_ylim(bounds)
# Ensure the aspect ratio is square
ax[1].set_aspect("equal", adjustable="box")

ax[1].plot(xi_real_test, xi_pred_test, '.')
ax[1].plot([0, 1], [0, 1], "r-",lw=2 ,transform=ax[1].transAxes)
ax[1].set(xlabel = '$\\xi$ / mol', ylabel = '$\\xi\mathregular{_{pred}}$ / mol')
ax[1].tick_params(direction = 'in') #, length = 20, width = 3)
ax[1].set_title('Test Data')


#plt.legend()
#fig.suptitle("Parity Plot")
plt.tight_layout()
plt.show()

#### Plot Fehler vs Epoch

In [ ]:
fig,ax = plt.subplots()
#fig.figsize=(12, 8)
ax.plot(train_MRE, label='train MRE')
ax.plot(test_MRE, label='test MRE')
plt.title("Mean Relative Error")
ax.set(xlabel = '$Epochs$ / 1', ylabel = '$MRE$ / mol') #Beschriftung Achsen; Kursiv durch $$; Index durch _{}
ax.tick_params(direction = 'in') #, length = 20, width = 3)

plt.legend()
plt.tight_layout()
plt.show()

#### Plot Loss vs Variable Properties

In [ ]:
mistake = []
param_T = []
param_p = []
param_x_H2 = []
param_x_N2 = []
param_x_NH3 = []
for X,y in train_dataloader:
    mistake = np.append(mistake, abs(y - net(X).detach().numpy()))
    param_T = np.append(param_T, X[:,0])
    param_p = np.append(param_p, X[:,1])
    param_x_H2 = np.append(param_x_H2, X[:,2])
    param_x_N2 = np.append(param_x_N2, X[:,3])
    param_x_NH3 = np.append(param_x_NH3, X[:,4])
    
# train_parameters, train_xi = next(iter(train_dataloader))
# y = abs(train_xi - net(train_parameters).detach().numpy())
# #[T, p ,x_H2, x_N2, x_NH3]
# x = [train_parameters[:,0], train_parameters[:,1], train_parameters[:,2], train_parameters[:,3], train_parameters[:,4]]

# print(param_T[0])
# print(param_T)
# print(mistake)

fig,ax = plt.subplots(1,3, figsize = (10, 5), gridspec_kw={'width_ratios': [2,2,3]})

ax[0].plot(param_T, mistake, '.', markersize = 2)
ax[0].set(xlabel = '$T$ / K', ylabel = '|$\\xi - \\xi\mathregular{_{pred}}$| / mol')
ax[0].tick_params(direction = 'in') #, length = 20, width = 3)

ax[1].plot(param_p, mistake, '.', markersize = 2)
ax[1].set(xlabel = '$p$ / bar', ylabel = '|$\\xi - \\xi\mathregular{_{pred}}$| / mol')
ax[1].tick_params(direction = 'in') #, length = 20, width = 3)

ax[2].plot(param_x_H2, mistake, '.', markersize = 2, label = '$x\mathregular{_{H_2}}$')
ax[2].plot(param_x_N2, mistake, '.', markersize = 2, label = '$x\mathregular{_{N_2}}$')
ax[2].plot(param_x_NH3, mistake, '.', markersize = 2, label = '$x\mathregular{_{NH_3}}$')
ax[2].set(xlabel = '$x\mathregular{_{NH_3}}$ / bar', ylabel = '|$\\xi - \\xi\mathregular{_{pred}}$| / mol')
ax[2].tick_params(direction = 'in') #, length = 20, width = 3)
ax[2].set

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
fig,ax = plt.subplots()
#fig.figsize=(12, 8)
ax.plot(train_accuracy, label='train accuracy')
ax.plot(test_accuracy, label='test accuracy')
plt.title("Train and Test Accuracy")
ax.set(xlabel = '$Epochs$ / 1', ylabel = '$Accuracy$ / %') #Beschriftung Achsen; Kursiv durch $$; Index durch _{}
ax.tick_params(direction = 'in') #, length = 20, width = 3)

plt.legend()
plt.tight_layout()
plt.show()

#### Debugging Hilfe

In [ ]:
# Anzeigen aller Input X und Output y Daten
for (X,y) in train_dataloader:
    print(X)
    print(y.reshape((-1,1)))
    print(net(X))
    break

#### Einblick in Netzwerk

In [ ]:
list(net.parameters()) # zeigt weights, biases, ...

In [ ]:
X = torch.rand((2,5))
print(X)
print(net(X))

In [ ]:
Lnorm = nn.LayerNorm(5)
Bnorm = nn.BatchNorm1d(5)


In [ ]:
for (X,y) in train_dataloader:
    print(X)
    #print(y.reshape((-1,1)))
    print(Bnorm(X).mean(dim=0))
    print(Bnorm(X))
    print(Lnorm(X))
    print((Lnorm(X.permute(0,2,1))).permute(0,2,1))
    print(Lnorm(X).mean(dim=0))
    print(Lnorm(X).mean(dim=1))


    break

#### Histogramme Verteilung von $xi$ und $x{_i}$

In [ ]:
#plt.hist(xi)
plt.hist(x_0[:,0],bins=100)
plt.hist(x_0[:,1],bins=100)

plt.show()